In [1]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams.update({"font.size": 12})
%matplotlib inline

In [2]:
!pip3 install flair

     |████████████████████████████████| 322 kB 13.1 MB/s 
     |████████████████████████████████| 48 kB 4.6 MB/s 
     |████████████████████████████████| 981 kB 48.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.4 MB 50.1 MB/s 
     |████████████████████████████████| 19.7 MB 320 kB/s 
     |████████████████████████████████| 61 kB 458 kB/s 
     |████████████████████████████████| 64 kB 2.7 MB/s 
     |████████████████████████████████| 788 kB 54.0 MB/s 
     |████████████████████████████████| 1.2 MB 55.2 MB/s 
     |████████████████████████████████| 62 kB 694 kB/s 
     |████████████████████████████████| 895 kB 70.6 MB/s 
     |████████████████████████████████| 3.3 MB 54.5 MB/s 
     |████████████████████████████████| 596 kB 77.2 MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9704 sha256=e9119277e8d08a2cb1aabb5ad0fca88fa58915880711a

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv("/content/drive/MyDrive/stocks/stocks_top_users_features.csv", sep=";")
df.tail(1)

,author,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,flairs,flairs_count,title_polarity,title_subjectivity,body_polarity,body_subjectivity,title_tokens,selftext_tokens,polarity
984,zeroskater45,16,Beyond Meat Cash Flow Issue? What is your favo...,"Hi,\n\nI noticed beyond meat has been a fairly...",2020-12-28 2020-12-04 2020-12-01 2020-09-03 20...,28 4 1 3 10 9 8 1 28 20 20 29 18 27 26 25,12 12 12 9 7 5 5 5 4 4 4 3 3 2 2 2,2020 2020 2020 2020 2020 2020 2020 2020 2020 2...,NaN,{},Ticker Discussion Discussion Question Discussi...,7.8125,125,1.0,16,0.5,8.0,"['Ticker', 'Discussion', 'Discussion', 'Questi...","{'Ticker': 2, 'Discussion': 5, 'Question': 1, ...",0.336443,0.457257,0.234165,0.509774,"['Beyond', 'Meat', 'Cash', 'Flow', 'Issue', '?...","['Hi', ',', '\n\n', 'I', 'noticed', 'beyond', ...",0.570608


# Embedding of titles by user

In [5]:
from flair.embeddings import WordEmbeddings, DocumentPoolEmbeddings
from flair.data import Sentence
from sklearn.manifold import TSNE
%matplotlib inline


## Visualize Embeddings for clustering

## Stacked Embeddings

In [ ]:
from flair.embeddings import  FlairEmbeddings
# initialize the word embeddings
glove_embedding = WordEmbeddings('glove')

# initialize the document embeddings, mode = mean
document_embeddings = DocumentPoolEmbeddings([glove_embedding])
# init Flair forward and backwards embeddings
flair_embedding_forward = FlairEmbeddings('news-forward')
flair_embedding_backward = FlairEmbeddings('news-backward')

2021-12-26 17:36:36,015 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmplmlea279


100%|██████████| 160000128/160000128 [00:06<00:00, 25998870.03B/s]

2021-12-26 17:36:42,480 copying /tmp/tmplmlea279 to cache at /root/.flair/embeddings/glove.gensim.vectors.npy


2021-12-26 17:36:42,804 removing temp file /tmp/tmplmlea279
2021-12-26 17:36:43,323 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /tmp/tmpegklw76a


100%|██████████| 21494764/21494764 [00:01<00:00, 14792396.61B/s]

2021-12-26 17:36:45,096 copying /tmp/tmpegklw76a to cache at /root/.flair/embeddings/glove.gensim


2021-12-26 17:36:45,132 removing temp file /tmp/tmpegklw76a


In [ ]:
from flair.embeddings import StackedEmbeddings

# create a StackedEmbedding object that combines glove and forward/backward flair embeddings
stacked_embeddings = StackedEmbeddings([
                                        glove_embedding,
                                        flair_embedding_forward,
                                        flair_embedding_backward,
                                       ])

In [ ]:
def get_stackedsentence_emb(text):
  sentence = Sentence(text)
  stacked_embeddings.embed(sentence)
  return sentence.embedding


# now check out the embedded tokens.
#for token in sentence:
#    print(token)
#    print(token.embedding)

In [ ]:
#df['title_emb_stacked'] = df['title'].apply(lambda x: get_stackedsentence_emb(x))
title_emb_stacked = df.loc[:50,'title'].apply(lambda x: get_stackedsentence_emb(x))

In [ ]:
from sklearn.manifold import TSNE
%matplotlib inline
def plot_embeddings(df_name,label_name,emb_col_name,total_points):

	all_users = []
	all_users_emb = []
	for index, row in df_name.iterrows():
		 #print(row["author"], row["title_emb"])
		 all_users.append(row[label_name])
		 all_users_emb.append(row[emb_col_name].numpy())
		 

	tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
	new_values = tsne_model.fit_transform(all_users_emb)

	x = []
	y = []
	for value in new_values:
		x.append(value[0])
		y.append(value[1])
			
	plt.figure(figsize=(16, 16)) 
	for i in range(len(x[0:total_points])):
		plt.scatter(x[i],y[i])
		plt.annotate(all_users[i],
						 xy=(x[i], y[i]),
						 xytext=(5, 2),
						 textcoords='offset points',
						 ha='right',
						 va='bottom')
	plt.show()

In [ ]:
#plot_embeddings(df,"author","title_emb_stacked",100)

In [ ]:
#plot_embeddings(df[df['score'] > 250],"author","title_emb_stacked",50)

## Count Years, Months 

In [6]:
from collections import Counter

In [7]:
df['year'] = df['year'].apply(lambda x: list(x.split(" ")))
df["years_count"] = df['year'].apply(lambda x: dict(Counter(list(x))))

In [8]:
df['month'] = df['month'].apply(lambda x: list(x.split(" ")))
df["months_count"] = df['month'].apply(lambda x: dict(Counter(list(x))))

In [9]:
df['date'] = df['date'].apply(lambda x: list(x.split(" ")))
df["dates_count"] = df['date'].apply(lambda x: dict(Counter(list(x))))

In [10]:
df.head(1)

,author,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,flairs,flairs_count,title_polarity,title_subjectivity,body_polarity,body_subjectivity,title_tokens,selftext_tokens,polarity,years_count,months_count,dates_count
0,-0-itisaplane,13,thoughts on dropbox ? Thinking of making anoth...,"currently trading at 20.72 US.\n\nhigh, median...","[2020-08-31, 2020-08-29, 2020-08-10, 2020-08-0...",31 29 10 5 26 24 23 22 22 22 21 21 12,"[8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7]","[2020, 2020, 2020, 2020, 2020, 2020, 2020, 202...",NaN,{},Discussion,25.538462,332,1.846154,24,0.897692,11.67,"['Discussion', '', '', '', '', '', '', '', '',...","{'Discussion': 1, '': 12}",0.066031,0.538518,0.104859,0.454719,"['thoughts', 'on', 'dropbox', '?', 'Thinking',...","['currently', 'trading', 'at', '20.72', 'US', ...",0.17089,{'2020': 13},"{'8': 4, '7': 9}","{'2020-08-31': 1, '2020-08-29': 1, '2020-08-10..."


In [11]:
df.to_csv("/content/drive/MyDrive/stocks/stocks_top_users_features.csv",sep=';')

## Are there differences between "old" users (pre-2021) and new ones?

In [12]:
# split dataset: pre and post hype 
#users_pre = df['year'].apply(lambda x:list(set(x).intersection(pre_years)))
users_post = df[['2021' in x for x in df['year'].values] and ['2020' not in x for x in df['year'].values] and ['2018' not in x for x in df['year'].values] and ['2019' not in x for x in df['year'].values]]
users_post.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 793 entries, 0 to 984
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   author              793 non-null    object 
 1   id                  793 non-null    int64  
 2   title               793 non-null    object 
 3   selftext            793 non-null    object 
 4   date                793 non-null    object 
 5   day                 793 non-null    object 
 6   month               793 non-null    object 
 7   year                793 non-null    object 
 8   tickers_            222 non-null    object 
 9   tickers_count_      793 non-null    object 
 10  link_flair_text     793 non-null    object 
 11  comments_mean       793 non-null    float64
 12  comments_sum        793 non-null    int64  
 13  score_mean          793 non-null    float64
 14  score_sum           793 non-null    int64  
 15  upvote_ratio_mean   793 non-null    float64
 16  upvote_r

In [17]:
users_post.to_csv("/content/drive/MyDrive/stocks/stocks_users_post-hype.csv",sep=';')

In [18]:
users_pre = df[['2021' not in x for x in df['year'].values]]
users_pre.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 283 entries, 0 to 984
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   author              283 non-null    object 
 1   id                  283 non-null    int64  
 2   title               283 non-null    object 
 3   selftext            283 non-null    object 
 4   date                283 non-null    object 
 5   day                 283 non-null    object 
 6   month               283 non-null    object 
 7   year                283 non-null    object 
 8   tickers_            53 non-null     object 
 9   tickers_count_      283 non-null    object 
 10  link_flair_text     283 non-null    object 
 11  comments_mean       283 non-null    float64
 12  comments_sum        283 non-null    int64  
 13  score_mean          283 non-null    float64
 14  score_sum           283 non-null    int64  
 15  upvote_ratio_mean   283 non-null    float64
 16  upvote_r

In [15]:
users_pre.head(1)

,author,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,flairs,flairs_count,title_polarity,title_subjectivity,body_polarity,body_subjectivity,title_tokens,selftext_tokens,polarity,years_count,months_count,dates_count
0,-0-itisaplane,13,thoughts on dropbox ? Thinking of making anoth...,"currently trading at 20.72 US.\n\nhigh, median...","[2020-08-31, 2020-08-29, 2020-08-10, 2020-08-0...",31 29 10 5 26 24 23 22 22 22 21 21 12,"[8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7]","[2020, 2020, 2020, 2020, 2020, 2020, 2020, 202...",NaN,{},Discussion,25.538462,332,1.846154,24,0.897692,11.67,"['Discussion', '', '', '', '', '', '', '', '',...","{'Discussion': 1, '': 12}",0.066031,0.538518,0.104859,0.454719,"['thoughts', 'on', 'dropbox', '?', 'Thinking',...","['currently', 'trading', 'at', '20.72', 'US', ...",0.17089,{'2020': 13},"{'8': 4, '7': 9}","{'2020-08-31': 1, '2020-08-29': 1, '2020-08-10..."


In [19]:
users_pre.to_csv("/content/drive/MyDrive/stocks/stocks_users_pre-hype.csv",sep=';')

In [20]:
users_pre_post = df[['2021' in x for x in df['year'].values] and ['2020' in x for x in df['year'].values] and ['2018' in x for x in df['year'].values] and ['2019' in x for x in df['year'].values]]
users_pre_post.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 1 to 979
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   author              192 non-null    object 
 1   id                  192 non-null    int64  
 2   title               192 non-null    object 
 3   selftext            192 non-null    object 
 4   date                192 non-null    object 
 5   day                 192 non-null    object 
 6   month               192 non-null    object 
 7   year                192 non-null    object 
 8   tickers_            51 non-null     object 
 9   tickers_count_      192 non-null    object 
 10  link_flair_text     192 non-null    object 
 11  comments_mean       192 non-null    float64
 12  comments_sum        192 non-null    int64  
 13  score_mean          192 non-null    float64
 14  score_sum           192 non-null    int64  
 15  upvote_ratio_mean   192 non-null    float64
 16  upvote_r

In [21]:
users_pre_post.head(1)

,author,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,flairs,flairs_count,title_polarity,title_subjectivity,body_polarity,body_subjectivity,title_tokens,selftext_tokens,polarity,years_count,months_count,dates_count
1,01Cloud01,27,Stock buyback monitoring website or tool $GT s...,I notice a lot or companies are issuing stock ...,"[2021-11-15, 2021-10-29, 2021-10-26, 2021-09-0...",15 29 26 6 30 21 22 3 3 1 23 15 1 7 20 15 12 1...,"[11, 10, 10, 9, 8, 8, 1, 1, 1, 1, 12, 11, 9, 8...","[2021, 2021, 2021, 2021, 2021, 2021, 2021, 202...","['SMG', 'GT']","{'SMG': 1, 'GT': 1}",Resources Company Discussion Company Analysis ...,7.407407,200,1.851852,50,0.573704,15.49,"['Resources', 'Company', 'Discussion', 'Compan...","{'Resources': 1, 'Company': 2, 'Discussion': 4...",0.299495,0.24899,0.129085,0.406224,"['Stock', 'buyback', 'monitoring', 'website', ...","['I', 'notice', 'a', 'lot', 'or', 'companies',...",0.42858,"{'2021': 10, '2020': 12, '2019': 4, '2018': 1}","{'11': 2, '10': 2, '9': 4, '8': 3, '1': 5, '12...","{'2021-11-15': 1, '2021-10-29': 1, '2021-10-26..."


In [22]:
users_pre_post.to_csv("/content/drive/MyDrive/stocks/stocks_users_pre-post_hype.csv",sep=';')

## Can we identify those that have posted (successful) DDs, Analysis, Company?

In [29]:
df['flairs'].values

array(["['Discussion', '', '', '', '', '', '', '', '', '', '', '', '']",
       "['Resources', 'Company', 'Discussion', 'Company', 'Analysis', 'Trades', '', 'Industry', 'Question', 'Ticker', 'Question', 'Discussion', 'Advice', 'Advice', 'Discussion', 'Discussion', 'Ticker', 'Question', 'Question', 'Ticker', 'Question', 'Question', '', 'Question', 'Question', '', '', '', '', '', 'Question', '', '']",
       "['', '', '', '', '', '', '', '', '', '', 'Discussion', '']",
       "['Advice', 'Industry', 'News', 'Industry', 'Discussion', 'Industry', 'Discussion', '', 'Off-Topic', 'Off-Topic', 'Ticker', 'Discussion', 'News', '', 'Ticker', 'Discussion', 'Ticker', 'Discussion', '', 'Ticker', 'Discussion', '', 'News', 'News', '', '', 'News', '', 'News', 'News', 'News', 'News', '', 'Ticker', 'Discussion', 'News', 'News', '', 'News', 'News', 'News', 'Ticker', 'Discussion', '', 'Ticker', 'Discussion', 'News', 'News', 'News', 'News', 'News', 'News', '', 'News', 'News', 'News', 'News', 'News', 'News',

In [30]:
## get all users with 'DD' flair and high upvotes, positive sentiment 
users_DD = df[['Advice' in x for x in df['flairs'].values] or ['Ticker' in x for x in df['flairs'].values]]
users_DD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497 entries, 1 to 982
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   author              497 non-null    object 
 1   id                  497 non-null    int64  
 2   title               497 non-null    object 
 3   selftext            497 non-null    object 
 4   date                497 non-null    object 
 5   day                 497 non-null    object 
 6   month               497 non-null    object 
 7   year                497 non-null    object 
 8   tickers_            140 non-null    object 
 9   tickers_count_      497 non-null    object 
 10  link_flair_text     497 non-null    object 
 11  comments_mean       497 non-null    float64
 12  comments_sum        497 non-null    int64  
 13  score_mean          497 non-null    float64
 14  score_sum           497 non-null    int64  
 15  upvote_ratio_mean   497 non-null    float64
 16  upvote_r

In [ ]:
users_DD.head(2)

In [24]:
users_DD.describe()

,id,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,title_polarity,title_subjectivity,body_polarity,body_subjectivity,polarity
count,985.000000,985.000000,985.000000,985.000000,985.000000,985.000000,985.000000,985.000000,985.000000,985.000000,985.000000,985.000000
mean,22.774619,28.519815,999.263959,7.581921,230.048731,0.770486,17.268071,0.128167,0.423134,0.096084,0.417602,0.224251
std,47.137620,41.971150,9776.052485,51.055797,2100.601050,0.326129,31.351345,0.158982,0.136608,0.064043,0.133405,0.181416
min,11.000000,0.000000,0.000000,1.000000,11.000000,0.000000,0.000000,-1.000000,0.000000,-0.337492,0.000000,-0.961706
25%,12.000000,9.000000,129.000000,1.000000,15.000000,0.636364,10.000000,0.035000,0.347563,0.060348,0.409974,0.111093
50%,15.000000,15.764706,273.000000,1.225806,23.000000,0.961538,12.350000,0.117949,0.423611,0.097497,0.446193,0.210298
75%,22.000000,32.727273,628.000000,2.176471,48.000000,0.999231,18.000000,0.209007,0.500000,0.131791,0.477965,0.326346
max,1016.000000,548.916667,297918.000000,1232.000000,53236.000000,1.000000,637.370000,1.000000,1.000000,0.357143,0.850000,1.183293


In [25]:
users_DD_top = users_DD[users_DD['score_mean'] > 60]
users_DD_top.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 27 to 940
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   author              18 non-null     object 
 1   id                  18 non-null     int64  
 2   title               18 non-null     object 
 3   selftext            18 non-null     object 
 4   date                18 non-null     object 
 5   day                 18 non-null     object 
 6   month               18 non-null     object 
 7   year                18 non-null     object 
 8   tickers_            3 non-null      object 
 9   tickers_count_      18 non-null     object 
 10  link_flair_text     18 non-null     object 
 11  comments_mean       18 non-null     float64
 12  comments_sum        18 non-null     int64  
 13  score_mean          18 non-null     float64
 14  score_sum           18 non-null     int64  
 15  upvote_ratio_mean   18 non-null     float64
 16  upvote_r

In [26]:
users_DD.to_csv("/content/drive/MyDrive/stocks/stocks_users_DD.csv",sep=';')

## Can we rank the users according to their expertise?

In [27]:
## rank users based on upvote ratio, num_comments etc.
## order by score_mean 
users_DD = users_DD.sort_values(by=['score_mean'],ascending=False)
users_DD.head()

,author,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,flairs,flairs_count,title_polarity,title_subjectivity,body_polarity,body_subjectivity,title_tokens,selftext_tokens,polarity,years_count,months_count,dates_count
661,hooman_or_whatever,26,Quit my Career to Become a Day Trader/Entrepre...,Hello all!\n\nFor those who don't know the bac...,"[2021-04-18, 2021-03-31, 2021-03-09, 2021-02-2...",18 31 9 26 25 24 16 16 9 3 3 2 25 22 14 10 9 8...,"[4, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, ...","[2021, 2021, 2021, 2021, 2021, 2021, 2021, 202...",NaN,{},Trades Company Analysis Company Discussion Co...,399.153846,10378,1232.000000,32032,0.978077,25.43,"['Trades', '', 'Company', 'Analysis', 'Company...","{'Trades': 1, '': 10, 'Company': 5, 'Analysis'...",0.094223,0.349148,0.089229,0.454340,"['Quit', 'my', 'Career', 'to', 'Become', 'a', ...","['Hello', 'all', '!', '\n\n', 'For', 'those', ...",0.183452,"{'2021': 15, '2020': 11}","{'4': 1, '3': 2, '2': 9, '1': 3, '12': 4, '11'...","{'2021-04-18': 1, '2021-03-31': 1, '2021-03-09..."
807,one8e4,18,Regulations not always Negative Are there othe...,"First will say, Not a supporter of socialism,\...","[2021-07-27, 2021-07-16, 2021-05-27, 2021-04-1...",27 16 27 16 2 8 3 31 31 28 28 14 4 16 12 12 6 6,"[7, 7, 5, 4, 3, 2, 2, 1, 1, 1, 1, 1, 1, 12, 12...","[2021, 2021, 2021, 2021, 2021, 2021, 2021, 202...",NaN,{},Industry Discussion Advice Request Advice Comp...,150.500000,2709,676.555556,12178,0.974444,17.54,"['Industry', 'Discussion', 'Advice', 'Request'...","{'Industry': 1, 'Discussion': 3, 'Advice': 6, ...",0.052333,0.407667,0.143329,0.574002,"['Regulations', 'not', 'always', 'Negative', '...","['First', 'will', 'say', ',', 'Not', 'a', 'sup...",0.195662,"{'2021': 13, '2020': 5}","{'7': 2, '5': 1, '4': 1, '3': 1, '2': 2, '1': ...","{'2021-07-27': 1, '2021-07-16': 1, '2021-05-27..."
676,ilai_reddead,22,Why I think tech stocks are a good buy Why I d...,This is going to be a slightly interesting pos...,"[2021-03-09, 2021-02-27, 2021-02-15, 2021-02-1...",9 27 15 13 13 13 11 2 1 1 1 30 29 29 28 28 28 ...,"[3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, ...","[2021, 2021, 2021, 2021, 2021, 2021, 2021, 202...",NaN,{},Meta,184.818182,4066,440.590909,9693,0.935909,20.59,"['', '', '', '', '', '', '', 'Meta', '', '', '...","{'': 21, 'Meta': 1}",0.044697,0.354545,0.096322,0.519839,"['Why', 'I', 'think', 'tech', 'stocks', 'are',...","['This', 'is', 'going', 'to', 'be', 'a', 'slig...",0.141019,{'2021': 22},"{'3': 1, '2': 10, '1': 11}","{'2021-03-09': 1, '2021-02-27': 1, '2021-02-15..."
27,AjaxFC1900,22,Janet Yellen to call for global minimum tax ra...,Janet Yellen will use her first major address ...,"[2021-04-05, 2021-02-23, 2021-02-23, 2021-02-1...",5 23 23 12 10 24 24 18 18 31 31 27 27 26 26 2 ...,"[4, 2, 2, 2, 2, 3, 2, 2, 2, 1, 1, 1, 1, 1, 1, ...","[2021, 2021, 2021, 2021, 2021, 2020, 2020, 202...",NaN,{},Off topic: Political Bullshit Industry News C...,309.363636,6806,335.954545,7391,0.221364,4.87,"['', 'Off', 'topic:', 'Political', 'Bullshit',...","{'': 17, 'Off': 1, 'topic:': 1, 'Political': 1...",0.086942,0.367358,0.078691,0.339481,"['Janet', 'Yellen', 'to', 'call', 'for', 'glob...","['Janet', 'Yellen', 'will', 'use', 'her', 'fir...",0.165633,"{'2021': 5, '2020': 10, '2019': 7}","{'4': 1, '2': 12, '3': 1, '1': 6, '12': 1, '5'...","{'2021-04-05': 1, '2021-02-23': 2, '2021-02-12..."
153,Floofyboy,11,"The ""crypt"" rule is a bit too strict VFV.TO vs...",Example: Some dude ask if coinbase is a good s...,"[2021-05-23, 2021-04-20, 2021-04-03, 2021-03-1...",23 20 3 12 9 8 4 3 27 20 20,"[5, 4, 4, 3, 3, 3, 3, 3, 2, 2, 2]","[2021, 2021, 2021, 2021, 2021, 2021, 2021, 202...",NaN,{},Meta Advice Request Advice Industry Discussion...,127.909091,1407,257.454545,2832,0.939091,10.33,"['Meta', 'Advice', 'Request', 'Advice', 'Indus...","{'Meta': 1, 'Advice': 7, 'Request': 4, 'Indust...",0.428571,0.467857,0.05

## Are the same users active in the other reviewed subreddits?

In [ ]:
## compare users with investing and stocks

# intersect top users of all three subreddits to find common users